In [1]:
from k1lib.imports import *

In [2]:
def mag(v): return math.sqrt((v**2).sum())
def norm(v): return v/mag(v)
def angleV(v1, v2):
    return math.acos(norm(v1) @ norm(v2)) * 180 / math.pi
def angleA(a1, a2, a3):
    x1 = s.x(a1); x2 = s.x(a2); x3 = s.x(a3)
    return angleV(x2 - x1, x2 - x3)
def magA(a1, a2):
    return mag(s.x(a1) - s.x(a2))

In [3]:
settings.mo.overOctet = True

In [4]:
t = k1lib.Wrapper(kcsv.cat("props.csv") | apply(op().replace("\ufeff", ""), 0) | apply(op().strip(), 2) | deref())
t() | display()

No    Formula   Name                                Molwt     Tfp (K)   Tb (K)   Tc (K)   Pc (bar)   Vc    Zc      Omega   Dipm   CpA     CpB      CpC          CpD         dHf       dGf       Eq   VpA        VpB       VpC        VpD        Tmin   Tmax    Lden    Tden   
182   C3H3F5    1,1,1,2,2-pentafluoropropane        134.047             255.7    380.1    31.4       273   0.271   0.308                                                                        0                                                                             
149   C2H3Cl3   1,1,1-trichloroethane               133.405   240       347.2    545      43                       0.217   1.7                                                                  1    -7.31317   2.04642   -3.77747   -0.45475   247    545     1.339   298    
152   C2H3F3    1,1,1-trifluoroethane               84.041    161.9     225.6    346.3    37.6       194   0.253   0.251   2.3    5.744   0.3141   -0.0002597   8.415E-08   -746100   -6792

In [6]:
names = k1lib.Wrapper(t() | columns()[:3] | sort() | deref())
names() | display(); names() | shape()

1    AlBr3   aluminum tribromide    
2    AlCl3   aluminum trichloride   
3    AlI3    aluminum triiodide     
4    Ar      argon                  
5    As      arsenic                
6    AsCl3   arsenic trichloride    
7    BBr3    boron tribromide       
8    BCl3    boron trichloride      
9    BF3     boron trifluoride      
10   BI3     boron triiodide        


(618, 3)

In [7]:
ids = names() | expandE(lambda s: (s, re.search(mo.parseM.cis_trans, s)), 2) | filt(op()!=None, 3) | cut(0) | deref() | aS(set) | aS(list) # "cis trans" at end
names() | filt(lambda s: "trans" in s or "cis" in s, 2) | ~inSet(ids, 0) | display(None), ids | shape() # "cis trans" not at end

33    F2N2      cis-difluorodiazine                
34    F2N2      trans-difluorodiazine              
141   C2H2Cl2   cis-1,2-dichloroethene             
142   C2H2Cl2   trans-1,2-dichloroethene           
526   C9H18     1,trans-3,5-trimethylcyclohexane   
557   C10H18    cis-decalin                        
558   C10H18    trans-decalin                      


(None, (22,))

In [8]:
passedMain = k1lib.Wrapper(names() | expandE(lambda s: (s, re.search(mo.parseM.main, s)), 2) | filt(op()!=None, 3) | cut()[:3] | deref()); passedMain() | display(); passedMain() | shape()

68    Xe        xenon                        
74    HF2N      difluoroamine                
87    CBrClF2   bromochlorodifluoromethane   
88    CBr2F2    dibromodifluoromethane       
89    CBrF3     trifluorobromomethane        
90    CClF3     chlorotrifluoromethane       
91    CCl2F2    dichlorodifluoromethane      
93    CCl3F     trichlorofluoromethane       
95    CD4       deutromethane                
101   CHClF2    chlorodifluoromethane        


(340, 3)

In [9]:
passedStrict = k1lib.Wrapper(names() | expandE(lambda s: (s, re.search(mo.parseM.fullStrictC, s)), 2) | filt(op()!=None, 3) | cut()[:3] | deref())
passed = k1lib.Wrapper(names() | expandE(lambda s: (s, re.search(mo.parseM.fullC, s)), 2) | filt(op()!=None, 3) | cut()[:3] | deref())
passed() | ~inSet(passedStrict()|cut(0)|aS(set), 0) | display(None) # does not pass strict, but does pass normal

68    Xe        xenon                           
95    CD4       deutromethane                   
176   C2H7NO    monoethanolamine                
177   C2H8N2    ethylenediamine                 
200   C3H6O     vinyl methyl ether              
245   C4H8O     vinyl ethyl ether               
267   C4H10O2   1,2-dimethoxyethane             
331   C6F14     perfluoro-2,3-dimethlybutane    
387   C6H14O    methyl amyl ether               
451   C8H10O    ethyl pheny ether               
544   C10H12    1,2,3,4-tetrahydronaphthalene   
576   C11H22    n-hexylcylopentane              


Recap:

In [10]:
[names(), passedMain(), passed(), passedStrict()] | shape(0).all() | deref()

[618, 340, 340, 328]

In [11]:
groupSamples = k1lib.Wrapper(passedStrict() | apply(mo.parseM.recognizeInit, 2) | cut(2) | cut(1) | joinStreams() | op().strip().all() | aS(set) | sort(numeric=False) | deref())
mainSamples = k1lib.Wrapper(passedStrict() | apply(mo.parseM.recognizeInit, 2) | cut(2) | cut(0) | apply(lambda x: x or "").all() | deref().all() | unique(0) | deref())
[passedStrict(), groupSamples(), mainSamples()] | shape(0).all() | deref()

[328, 115, 112]

Quality assurance:

In [12]:
assert passedStrict() | apply(mo.parseM.recognizeInit, 2) | instanceOf(str, 2) | shape(0) == 0 # every passedStrict MUST run through recognize fine
assert groupSamples() | apply(lambda s: re.fullmatch(mo.parseM.branchC, s)) | filt(op()==None) | shape(0) == 0
assert mainSamples()  | apply(lambda s: re.fullmatch(mo.parseM.mainC, s[0])) | filt(op()==None) | shape(0) == 0

92% chance of main recognition! Other cases are sort of weird, but that's quite outstanding!

In [13]:
failedRecMain = mainSamples() | apply(mo.parseM.recognizeMain) | ~instanceOf(mo.Atom) | deref()
ratio = 100 - (failedRecMain | shape(0)) / (mainSamples() | shape(0)) * 100; ratio, failedRecMain

(92.85714285714286,
 ['amine',
  'ether',
  'pyridine',
  'cyclopentanone',
  'benzene',
  'cyclohexanone',
  'benzaldehyde',
  'naphthalene'])

In [14]:
assert ratio > 90

98% chance of group recognition!

In [15]:
failedRecGroup = groupSamples() | apply(partial(mo.parseM.recognizeGroup, mainA=None)) | ~instanceOf(mo.Atom) | deref(); failedRecGroup | headOut()
ratio = 100 - (failedRecGroup | shape(0)) / (groupSamples() | shape(0)) * 100; ratio

nitro
perfluoro


98.26086956521739

In [16]:
assert ratio > 98

73% chance of end-to-end recognition. Not the greatest in the world, but still quite awesome!

In [17]:
fullyRecognized = k1lib.Wrapper(passedStrict() | apply(partial(mo.parse, quiet=True), 2) | instanceOf(mo.Atom, 2) | deref())
ratio = (fullyRecognized() | shape(0)) / (passedStrict() | shape(0)) * 100; ratio, fullyRecognized() | shape()

(73.47560975609755, (241, 3))

In [18]:
assert ratio > 73

39% recognition chance over the entire dataset.

In [19]:
ratio = (fullyRecognized() | shape(0)) / (t() | shape(0)) * 100; ratio

38.933764135702745

In [20]:
assert ratio > 38

Self-consistency check:

In [44]:
try:
    for _i, _a, _b in zip(*(fullyRecognized() | apply(lambda a: a.empirical(), 2) | transpose() | deref())):
        if not mo.sameEmpirical(_a, _b):
            raise RuntimeError(f"{_i}, {_a}, {_b}")
except RuntimeError as e:
    warnings.warn(str(e))

/tmp/ipykernel_639991/3375504456.py:6: UserWarning: 452, C8H10O, C6H6O
  warnings.warn(str(e))


Everything else not recognized:

In [24]:
t() | ~inSet(passedStrict()|cut(0)|aS(set)|aS(list), 0) | display(None)

No    Formula    Name                                 Molwt     Tfp (K)   Tb (K)   Tc (K)   Pc (bar)   Vc      Zc      Omega    Dipm       CpA       CpB           CpC            CpD            dHf        dGf        Eq   VpA         VpB           VpC          VpD          Tmin   Tmax    Lden     Tden   
544   C10H12     1,2,3,4-tetrahydronaphthalene        132.206   242       480.7    719      35.1                       0.303                                                                     2760       167100     3    9.5883      4009.49       -64.89                    365    500     0.973    293    
471   C8H16      1,2,4-trimethylcyclopentane-c,c,t    112.216             391      579      29                         0.277                                                                                           3    9.1341      3073.95       -54.2                     283    418                     
472   C8H16      1,2,4-trimethylcyclopentane-c,t,c    112.216             382.4    571  